In [0]:
import pandas as pd
import numpy as np
from keras.models import Model
import tensorflow as tf
import re
from nltk.corpus import stopwords
import time
import numpy as np
import io
import nltk
import pickle
from pickle import dump, load
from google.colab import files
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import tensor_array_ops
print('TensorFlow Version: {}'.format(tf.__version__))
nltk.download('stopwords')

Using TensorFlow backend.


    100% |████████████████████████████████| 993kB 14.5MB/s 
TensorFlow Version: 1.13.1
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# load weights
link='https://drive.google.com/open?id=1DJFvHYEQY_cAKh37yTY80iOzviRTsimw'
fluff, id = link.split('=')
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('best_model.h5.data-00000-of-00001')

link='https://drive.google.com/open?id=1kV4hl4ujLDysxchJJY94QsZPOFWXBEsM'
fluff, id = link.split('=')
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('best_model.h5.meta')
import nltk
nltk.download('stopwords')


1DJFvHYEQY_cAKh37yTY80iOzviRTsimw
1kV4hl4ujLDysxchJJY94QsZPOFWXBEsM
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:

link='https://drive.google.com/open?id=14S3NOL1Cc6NT214Fl0GIbjsu9NjXEFm1'
fluff, id = link.split('=')
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('vocab_to_int.pkl')
vocab_to_int = load(open('vocab_to_int.pkl', 'rb'))
print(type(vocab_to_int))


14S3NOL1Cc6NT214Fl0GIbjsu9NjXEFm1
<class 'dict'>


In [0]:

link='https://drive.google.com/open?id=19UT1XqkcR7kV6WIO3u9FYmFOqFy-FdyI'
fluff, id = link.split('=')
print (id)
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('int_to_vocab.pkl')
int_to_vocab = load(open('int_to_vocab.pkl', 'rb'))
print(type(int_to_vocab))

19UT1XqkcR7kV6WIO3u9FYmFOqFy-FdyI
<class 'dict'>


In [0]:

contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}
def clean_text(text, remove_stopwords=True):
    '''Remove unwanted characters, stopwords, and format the text to create fewer nulls word embeddings'''

    # Convert words to lower case
    text = text.lower()

    # Replace contractions with their longer forms
    if True:
        text = text.split()
        new_text = []
        for word in text:
            if word in contractions:
                new_text.append(contractions[word])
            else:
                new_text.append(word)
        text = " ".join(new_text)

    # Format words and remove unwanted characters
    text = re.sub(r'https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    text = re.sub(r'\<a href', ' ', text)
    text = re.sub(r'&amp;', '', text)
    text = re.sub(r'[_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'>', ' ', text)
    text = re.sub(r'<br', ' ', text)
    text = re.sub(r'\'', ' ', text)

    # Optionally, remove stop words
    if remove_stopwords:
        text = text.split()
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
        text = " ".join(text)

    return text

In [0]:

batch_size = 64

def text_to_seq(text):
    '''Prepare the text for the model'''
    
    text = clean_text(text)
    return [vocab_to_int.get(word, vocab_to_int['<UNK>']) for word in text.split()]

In [0]:
input_sentences=["wife use everyday dinners crystal clean white finishing salt 33 lower sodium table salt 2 magnesium calcium super healthy used water workouts little pinch good electrolyte boost big fan product honestly cannot recommend enough reminds higher quality fleur de sel",
              "worst poduct, dont buy,bad food product i did not like it ",
                  "bad food product",
                 "Great taffy at a great price.There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.",
                 "Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product it as Jumbo bad worst.",
                 " bought local farmer specialty market thoroughly impressed affinity cheeses especially variety made easier pick quality product rich flavorful",
                 "excellent product long lasting hard candy sugar free great taste without funny taste sugar free candies"
                 ,"far delicious garlic ever tasted kids love go like candy happy good us"
                 
]
 
generagte_summary_length=[]
for i in range(len(input_sentences)):
        generagte_summary_length.append(10)

texts = [text_to_seq(input_sentence) for input_sentence in input_sentences]
checkpoint = "./best_model.h5"
if type(generagte_summary_length) is list:
    if len(input_sentences)!=len(generagte_summary_length):
        raise Exception("[Error] makeSummaries parameter generagte_summary_length must be same length as input_sentences or an integer")
    generagte_summary_length_list = generagte_summary_length
else:
    generagte_summary_length_list = [generagte_summary_length] * len(texts)
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)
    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    text_length = loaded_graph.get_tensor_by_name('text_length:0')
    summary_length = loaded_graph.get_tensor_by_name('summary_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
    #Multiply by batch_size to match the model's input parameters
    for i, text in enumerate(texts):
        generagte_summary_length = generagte_summary_length_list[i]
        answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                          summary_length: [generagte_summary_length], #summary_length: [np.random.randint(5,8)], 
                                          text_length: [len(text)]*batch_size,
                                          keep_prob: 1.0})[0] 
        # Remove the padding from the summaries
        pad = vocab_to_int["<PAD>"] 
        print('- Review:\n\r {}'.format(input_sentences[i]))
        print('- Summary:\n\r {}\n\r\n\r'.format(" ".join([int_to_vocab[i] for i in answer_logits if i != pad])))

INFO:tensorflow:Restoring parameters from ./best_model.h5
- Review:
 wife use everyday dinners crystal clean white finishing salt 33 lower sodium table salt 2 magnesium calcium super healthy used water workouts little pinch good electrolyte boost big fan product honestly cannot recommend enough reminds higher quality fleur de sel
- Summary:
 great crystal


- Review:
 worst poduct, dont buy,bad food product i did not like it 
- Summary:
 do not buy these bags from amazon com


- Review:
 bad food product
- Summary:
 not happy


- Review:
 Great taffy at a great price.There was a wide assortment of yummy taffy.  Delivery was very quick.  If your a taffy lover, this is a deal.
- Summary:
 great product


- Review:
 Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product it as Jumbo bad worst.
- Summary:
 what is wrong with the product not size


- Review:
  bought lo